In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U
!pip install transformers datasets sentencepiece tokenizers segeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 844.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a42983ffffe9c0468654b323f3db2d614e62b4f72b96d06c7ebe18f62948cb84
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import accelerate
import torch
import numpy as np


In [ ]:
#load dataset
marathi_dataset = load_dataset('ai4bharat/naamapadam', 'mr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/455248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1080 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2300 [00:00<?, ? examples/s]

###### Below Some cells are written to understand the dataset, its properties, how many columns, how much data is there

In [ ]:
marathi_dataset


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 455248
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1080
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2300
    })
})

In [ ]:
marathi_dataset.shape

{'train': (455248, 2), 'test': (1080, 2), 'validation': (2300, 2)}

In [ ]:
type(marathi_dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
marathi_dataset['train'][0:2]

{'tokens': [['विद्यापीठ',
   '/',
   'राज्य',
   'सरकार',
   '/',
   'केंद्र',
   'सरकार',
   '/',
   'संबंधित',
   'संस्थांचे',
   'नियम'],
  ['महाराष्ट्रात', 'कोरोनाचा', 'सर्वाधिक', 'कोरोना', 'रुग्ण', 'आहेत', '.']],
 'ner_tags': [[0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0, 0]]}

In [ ]:
marathi_dataset['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

###### Below some lines are just to understand some of the problems that were occuring after tokenization (No need to run these cells)

In [ ]:
#Import tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

In [ ]:
#extracting one sentence from dataset to see if tokenization works or not. If works then is there any problem or not
example_text=marathi_dataset['train'][0]
print("Example text= ",example_text)

#tokenizing the sentence
tokenized_input=tokenizer(example_text['tokens'],is_split_into_words=True)
print("After Tokenization=",tokenized_input)

tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])  #toconvert ids to tokens for viewing purpose


print("\nOriginal Sentence: ", example_text['tokens'])
print("Sentence after tokenization: ",tokens)
print("Total ner_tags for our original sentence: ", len(example_text['ner_tags']))
print("Total tokens generated after tokenization: ",len(tokenized_input['input_ids']))

print("\nThis is a problem. When I did tokenization then number of tokens increased but ner_tags remained same")
print("Reason is, even if tokens are already in the form of words in example_text, tokenizer can divide these words into subwords also")
print("This results in increase in number of tokens. Now to resolve this problem, we need to depend on word_id, mentioned in cell below")


Example text=  {'tokens': ['विद्यापीठ', '/', 'राज्य', 'सरकार', '/', 'केंद्र', 'सरकार', '/', 'संबंधित', 'संस्थांचे', 'नियम'], 'ner_tags': [0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0]}
After Tokenization= {'input_ids': [101, 53836, 48559, 59551, 10824, 24537, 120, 17264, 39425, 120, 55414, 39425, 120, 81205, 13036, 76008, 14870, 566, 48869, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Original Sentence:  ['विद्यापीठ', '/', 'राज्य', 'सरकार', '/', 'केंद्र', 'सरकार', '/', 'संबंधित', 'संस्थांचे', 'नियम']
Sentence after tokenization:  ['[CLS]', 'वि', '##दय', '##ाप', '##ी', '##ठ', '/', 'राजय', 'सरकार', '/', 'कदर', 'सरकार', '/', 'सबध', '##ित', 'ससथा', '##च', 'न', '##ियम', '[SEP]']
Total ner_tags for our original sentence:  11
Total tokens generated after tokenization:  20

This is a problem. When I did tokenization then number of tokens increased but ner_tags remained same
Reason is, e

In [ ]:
tokens=tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
print(tokens)
word_ids=tokenized_input.word_ids()
print(word_ids)
print("This means even if word is subdivided into more tokens, word id keeps track of the word from which the token is generated")



['[CLS]', 'वि', '##दय', '##ाप', '##ी', '##ठ', '/', 'राजय', 'सरकार', '/', 'कदर', 'सरकार', '/', 'सबध', '##ित', 'ससथा', '##च', 'न', '##ियम', '[SEP]']
[None, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, None]
This means even if word is subdivided into more tokens, word id keeps track of the word from which the token is generated


###### Defining Some important functions (Run these cells)

Now the cls and SEP are two extra(special) tokens that are mapped to 'None. To resolve this, just map these tokens to value -100. Because -100 is index which is ignored by pytorch during training. And labels(ner_tags) of all other tokens will be mapped to label(ner_tag) of word that they come from

So the function below will do two jobs:
1. Assign special tokens to  label(ner_tag) '-100'
2. Masking of subword representation after first subword

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

def tokenize_and_align_label(example, label_all_tokens=True):  #input is example from dataset and a boolean argument
  tokenized_input= tokenizer(example['tokens'], truncation=True, is_split_into_words=True)      #tokenizing the input
  #print("Before aligning the labels:",tokenized_input)
  #e.g. ['[CLS]', 'वि', '##दय', '##ाप', '##ी', '##ठ', '/', 'राजय', 'सरकार', '/', 'कदर', 'सरकार', '/', 'सबध', '##ित', 'ससथा', '##च', 'न', '##ियम', '[SEP]']

  labels= []   #This list will contain correct labels for tokenized input of the example
  for i, label in enumerate(example['ner_tags']):
    word_ids = tokenized_input.word_ids(batch_index=i)    #Extracting word id of tokenized input with batch_index i
    #here word_ids() function call will return list of mapping the tokens to their actual word in initial sentence.
    #It returns the list indicating the word corresponding to each token. For example mentioned in comment above word_ids list is [None, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, None]

    previous_word_idx =None
    label_ids=[]              #to set labels for tokenized input
    for word_idx in word_ids:
      if word_idx is None:    #then just setting it as -100 for special tokens
        label_ids.append(-100)
      elif word_idx != previous_word_idx:   #it means this token has came from new word. And it has its own label so just add label of that word
        label_ids.append(label[word_idx])
      else:                   #
        #to take care of sub_words which has same word_idx
        #So I am just setting those also as -100. But only if label_all_tokens==False
        label_ids.append(label[word_idx] if label_all_tokens else -100)
      previous_word_idx= word_idx

    labels.append(label_ids)      #after completing inner loop just appending labels_ids to overall labels

  #Now all labels are calculated. So I am just inserting new 'labels' into tokenized_input. It already had 'input_ids','attension_mask', 'token_type_ids' as keys.
  #I have printed tokenized input of one of the example in somewhere above cells
  tokenized_input['labels'] = labels
  return tokenized_input


tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

###### No need to run below two cells

In [ ]:
q= tokenize_and_align_label(marathi_dataset['train'][0:1])      #Just calling above function to check whether it is working or not
print("\nAfter Aligning the labels:",q)


After Aligning the labels: {'input_ids': [[101, 53836, 48559, 59551, 10824, 24537, 120, 17264, 39425, 120, 55414, 39425, 120, 81205, 13036, 76008, 14870, 566, 48869, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [ ]:
#just to view tokens and labels side by side
for token, label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
  print(f"{token:<20} {label}")

[CLS]                -100
वि                   0
##दय                 0
##ाप                 0
##ी                  0
##ठ                  0
/                    0
राजय                 0
सरकार                0
/                    0
कदर                  3
सरकार                4
/                    0
सबध                  0
##ित                 0
ससथा                 0
##च                  0
न                    0
##ियम                0
[SEP]                -100


###### Performing Tokenization. Defining model and training arguments

In [ ]:

tokenized_dataset=marathi_dataset.map(tokenize_and_align_label,batched=True)

Map:   0%|          | 0/455248 [00:00<?, ? examples/s]

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Map:   0%|          | 0/2300 [00:00<?, ? examples/s]

In [ ]:
#Defining Model
model=AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER", num_labels=7)       #num labels = 7 because we have 7 classes

#Defining Tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

#data collator actually automatically pads the model inputs in a batch to the length of longest example. These bypass the need to set a global maximum sequence length
data_collator = DataCollatorForTokenClassification(tokenizer)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
label_list= marathi_dataset['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [ ]:
metric=load_metric("seqeval")

<ipython-input-17-e52c73930dac>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


###### calculating metric on single example i.e. variable example. Just to check that metric.compute is working or not (No need to run these cells)

In [ ]:
example=marathi_dataset['train'][0]
print(example['tokens'])
labels =[label_list[i] for i in example['ner_tags']]    #extracting ner tags of that example
print(labels)

['विद्यापीठ', '/', 'राज्य', 'सरकार', '/', 'केंद्र', 'सरकार', '/', 'संबंधित', 'संस्थांचे', 'नियम']
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']


In [ ]:
metric.compute(predictions=[labels],references=[labels])      #I am passing same list in predictions and references both just to check whether it it working or not

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

###### Actual Compute metric method for entire dataset (Run below cells)

In [ ]:
metric=load_metric("seqeval")

def compute_metrics(eval_preds):
  pred_logits, labels =eval_preds
  pred_logits = np.argmax(pred_logits,axis=2) #logits and probabilities are in the same order, Hence I am not applying the softmax
  true_predictions = [
      [label_list[eval_preds] for (eval_preds,l) in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)
   ]  #as I need to pass a list in prediction argument in metric.compute
  true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
  results=metric.compute(predictions=true_predictions, references=true_labels)

  return {
      "precision" : results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

  # # Unpack nested dictionaries
  # final_results = {}
  # for key, value in results.items():
  #     if isinstance(value, dict):
  #         for n, v in value.items():
  #             final_results[f"{key}_{n}"] = v
  #     else:
  #         final_results[key] = value
  # return final_results


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


##### Model Configurations

In [ ]:
#Defining Training arguments parameters
args= TrainingArguments(
    output_dir="/content/drive/MyDrive/Mtech/trained_model/ner_model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    logging_dir="/content/drive/MyDrive/Mtech/loggingDirectory/logs",
    logging_strategy="steps",
    logging_steps=2000,  # Log every 1000 steps
    save_strategy="epoch",
    save_steps=2000,
)

In [ ]:
#In this trainer I am tokenizing some part of dataset
#Trying to get few of the samples from train
#then using these samples in trainer.


train_dataset = marathi_dataset["train"].select(range(100000))
train_dataset = train_dataset.map(
    tokenize_and_align_label,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
eval_dataset = marathi_dataset["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_label,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)


## Training the model (I have already trained the model. So do not execute cells which are under this heading)

In [ ]:
#I am commenting this line to take precaution from training the model again
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.246000,0.251865,0.804582,0.817766,0.811121,0.919107
2,0.203400,0.252990,0.804687,0.823286,0.813880,0.919772
3,0.157700,0.277835,0.804333,0.819664,0.811926,0.918397
4,0.118800,0.309569,0.802171,0.815869,0.808962,0.919137
5,0.091100,0.348613,0.802191,0.814575,0.808336,0.918896


TrainOutput(global_step=31250, training_loss=0.16794418310546874, metrics={'train_runtime': 7825.4282, 'train_samples_per_second': 63.894, 'train_steps_per_second': 3.993, 'total_flos': 1.911701640720221e+16, 'train_loss': 0.16794418310546874, 'epoch': 5.0})

Mounted at /content/drive


In [ ]:

model.save_pretrained('/content/drive/MyDrive/Mtech/SavedModel/ner_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Mtech/SavedModel/tokenizer')

('/content/drive/MyDrive/Mtech/SavedModel/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Mtech/SavedModel/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Mtech/SavedModel/tokenizer/vocab.txt',
 '/content/drive/MyDrive/Mtech/SavedModel/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Mtech/SavedModel/tokenizer/tokenizer.json')

In [ ]:
training_logs = trainer.state.log_history
print("Training Loss= ",training_logs[-1]['train_loss'])

Training Loss=  0.16794418310546874


## Training of IndicNER model on Marathi dataset is completed.
While saving the model into /content/drive/MyDrive/Mtech/SavedModel/ner_model folder,  only saved config.json file got created. It couldn't save 'pytorch_model.bin' file. Hence I couldn't import model from this savedModel folder

## But As I have saved checkpoints while training the model.
We can import the model as well as tokenizer from the last checkpoint. All the checkpoints are stored in /content/drive/MyDrive/Mtech/trained_model/ner_model directory.
In this case last checkpoint was 31250 hence our model and tokenizer is present in  /content/drive/MyDrive/Mtech/trained_model/ner_model/checkpoint-31250 folder

So, if I want to use my fine_tuned_model, just write below lines


tokenizer=AutoTokenizer.from_pretrained('/content/drive/MyDrive/Mtech/trained_model/ner_model/checkpoint-31250')
model=AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/Mtech/trained_model/ner_model/checkpoint-31250')

# Calculating performance parameters is done in another notebook named 'Testing_IndicNER'

In [ ]:
#This code is completed